# **Installation**

In [ ]:
# !pip install pycuda
# Package to run the reconstruction on the graphics card -> accelerates it A LOT

In [ ]:
# !curl --create-dirs -O --output-dir "pynx" http://ftp.esrf.fr/pub/scisoft/PyNX/pynx-latest.tar.bz2
# download the PyNx (European Synchrotron Radiation Facility, Grenoble, France) installation files - https://pynx.esrf.fr/en/latest/

In [ ]:
# !pip install "pynx/pynx-latest.tar.bz2"[cuda]
# install the PyNx package

# **Import GPU tools**

In [ ]:
import pycuda
import pycuda.autoinit
from pycuda.tools import make_default_context
c = make_default_context()
d = c.get_device()
d.name()

# **Read in image**

In [ ]:
import os

Basepath = os.path.normpath(r'/nfs/ruche/nanoscopium-users/20240588/2025/Shutdown6/2025-01-16')
Outputpath = os.path.normpath(r'/nfs/ruche/nanoscopium-users/20240588/published-data/data')
os.chdir(Basepath)
print('data:    ', os.getcwd())

print('files: ')
for file in os.listdir():
    print(file, end=', ')

In [ ]:
import h5py
import numpy as np

raw_file = "flyscan_16401-0001.nxs"
scan = "flyscan_16401"

# Open the NeXus file
with h5py.File(raw_file, 'r') as f:
    # List all groups
    print("Keys: %s" % f.keys())
    # Get the data
    data = np.array(f.get(scan+"/scan_data/Image_merlin_image"))
    print(np.shape(data))  # Print the data


In [ ]:
import matplotlib.pyplot as plt

image = np.sum(data, axis=(0,1))
plt.imshow(image, vmin=0,vmax=1)
plt.colorbar()

In [ ]:
from pynx.cdi import cdi
from scipy.fft import fftshift

# save the input in the needed format
pixel_size_detector = 55e-6
wavelength = 0.155e-9
detector_distance = 5.1

filename = os.path.join(Outputpath,'_'.join([raw_file.split('.')[0], 'input', 'CDI'])+'.hdf5')
cdi.save_cdi_data_cxi(filename, iobs=image, pixel_size_detector=pixel_size_detector, wavelength=wavelength, detector_distance=detector_distance)

# **Reconstruction**

In [ ]:
from pynx.cdi import    CDI, InitFreePixels, AutoCorrelationSupport, ShowCDI, ScaleObj, \
                        InitObjRandom, SupportUpdate, RAAR, ER, ML, HIO
from scipy.fft import fftshift
import h5py

In [ ]:
# open the prev. saved input file

File_Preface = 'CDI_data'
filename = os.path.join(Outputpath,'_'.join([raw_file.split('.')[0], 'input', 'CDI'])+'.hdf5')

# Extract data
cxi = h5py.File(filename, 'r')
if '/entry_1/instrument_1/source_1/energy' in cxi:
    nrj = cxi['/entry_1/instrument_1/source_1/energy'][()] / 1.60218e-16
    wavelength = 12.384 / nrj * 1e-10
    print("  CXI input: Energy = %8.2fkeV" % nrj)
else:
    wavelength = None
if '/entry_1/instrument_1/detector_1/distance' in cxi:
    detector_distance = cxi['/entry_1/instrument_1/detector_1/distance'][()]
    print("  CXI input: detector distance = %8.2fm" % detector_distance)
else:
    detector_distance = None
if '/entry_1/instrument_1/detector_1/x_pixel_size' in cxi:
    pixel_size_detector = cxi['/entry_1/instrument_1/detector_1/x_pixel_size'][()]
    print("  CXI input: detector pixel size = %8.2fum" % (pixel_size_detector * 1e6))
else:
    pixel_size_detector = None
print("  CXI input: loading iobs")
if 'entry_1/instrument_1/detector_1/data' in cxi:
    iobs = cxi['entry_1/instrument_1/detector_1/data'][()].astype(np.float32)
else:
    iobs = cxi['entry_1/data_1/data'].value.astype(np.float32)
# This is the detector mask
if 'entry_1/instrument_1/detector_1/mask' in cxi:
    mask = cxi['entry_1/instrument_1/detector_1/mask'][()].astype(np.int8)
    mask = mask.reshape(np.shape(iobs))
    nb = mask.sum()
    print("  CXI input: loading mask, with %d pixels masked (%6.3f%%)" % (nb, nb * 100 / mask.size))
else:
    mask = None

In [ ]:
# Create & initialise the CDI object
cdi = CDI(fftshift(iobs), obj=None, mask=mask, wavelength=wavelength, pixel_size_detector=pixel_size_detector)

cdi = InitFreePixels() * cdi

# Init the object support & a random object
cdi = AutoCorrelationSupport(threshold=0.03) * cdi
cdi = ShowCDI(fig_num=1) * ScaleObj() * InitObjRandom(src="support",amin=0.8,amax=1, phirange=0.0) * cdi

# Initial scaling of the object [ only useful if there are masked pixels !]
cdi = ScaleObj(method='F') * cdi

In [ ]:
# Define the support - which is crutial for the convergence
# the "threshold_relative" should be changed:
# 1. does converge to a single pixel
# 2. does converge to noise on the whole image
sup = SupportUpdate(threshold_relative=0.03, method='rms')#, smooth_width=(2,0.5,600))
plt.figure()

# Run the phase retrieval
# runs different methods 'RAAR' and 'ER' - usualy shows a good convergence
# quadration defines the number of iterations
cdi = ( RAAR(beta=0.9, calc_llk=50, show_cdi=50)**10)**60 * cdi
cdi = (sup * ER(calc_llk=50, show_cdi=50)**20)**10 * cdi

In [ ]:
 # save results to hdf files (all data + metadata)
filename = os.path.join(Outputpath,'_'.join([raw_file.split('.')[0],'Output_obj', 'CDI'])+'.hdf5')
cdi.save_obj_cxi(filename)
filename = os.path.join(Outputpath,'_'.join([raw_file.split('.')[0],'Output_data', 'CDI'])+'.hdf5')
cdi.save_data_cxi(filename)

In [ ]:
# save results - just to images
from PIL import Image

# object domain
filename = os.path.join(Outputpath,'_'.join([raw_file.split('.')[0],'Output_obj', 'CDI'])+'.tiff')

im = Image.fromarray(np.abs(cdi.get_obj(shift=True)))
im.save(filename)

# fourier domain
filename = os.path.join(Outputpath,'_'.join([raw_file.split('.')[0],'Output_data', 'CDI'])+'.tiff')

im = Image.fromarray(cdi.get_iobs(shift=True))
im.save(filename)